In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm

In [2]:
main_df = pd.DataFrame()

In [3]:
xls = pd.ExcelFile('./data/2016/Jan 2016.xlsx')
xls.sheet_names

['visibility', 'Dew pt', 'QFE', 'cloud', 'Temperature']

# For visibility data

In [4]:
df = pd.read_excel(xls, sheet_name='visibility', skiprows=3)
df.head()

,Day/Time(UTC),05:45:00,08:45:00,11:45:00,14:45:00,17:45:00
0,1,6,7.0,9,8,7
1,2,6,7.0,7,8,7
2,3,6,7.0,8,8,6
3,4,6,7.0,7,7,7
4,5,6,6.0,6,7,7


In [5]:
main_df = pd.concat([main_df, df], axis=1)
main_df.head()

,Day/Time(UTC),05:45:00,08:45:00,11:45:00,14:45:00,17:45:00
0,1,6,7.0,9,8,7
1,2,6,7.0,7,8,7
2,3,6,7.0,8,8,6
3,4,6,7.0,7,7,7
4,5,6,6.0,6,7,7


# For dew point

In [6]:
df = pd.read_excel(xls, sheet_name='Dew pt', skiprows=3)
df.head()

,Day/Time(UTC),05:45:00,08:45:00,11:45:00,14:45:00,17:45:00
0,1,7.4,8.4,7.8,8.8,10.3
1,2,7.9,8.8,8.7,9.8,10.2
2,3,7.7,8.0,9.7,9.8,9.4
3,4,8.4,8.6,9.8,9.2,9.4
4,5,7.1,9.2,9.5,10.7,11.2


In [7]:
df = df.set_index('Day/Time(UTC)').stack().reset_index().set_index('Day/Time(UTC)')
df.index = pd.to_datetime([f'Jan {x}, 2016 {y}' for x, y in zip(df.index, df['level_1'])])
df.drop('level_1', axis=1, inplace=True)
df.columns = ['dew_pt']
df.head()

,dew_pt
2016-01-01 05:45:00,7.4
2016-01-01 08:45:00,8.4
2016-01-01 11:45:00,7.8
2016-01-01 14:45:00,8.8
2016-01-01 17:45:00,10.3


In [8]:
main_df['dew_pt'] = df['dew_pt']
main_df.head()

,Day/Time(UTC),05:45:00,08:45:00,11:45:00,14:45:00,17:45:00,dew_pt
0,1,6,7.0,9,8,7,NaN
1,2,6,7.0,7,8,7,NaN
2,3,6,7.0,8,8,6,NaN
3,4,6,7.0,7,7,7,NaN
4,5,6,6.0,6,7,7,NaN


# For QFE

In [9]:
df = pd.read_excel(xls, sheet_name='QFE', skiprows=3)
df.head()

,Day/Time(UTC),05:45:00,08:45:00,11:45:00,14:45:00,17:45:00
0,1,927.0,929.3,930.0,927.0,927.3
1,2,927.3,929.1,928.4,925.7,924.6
2,3,924.8,926.5,927.3,924.2,923.6
3,4,925.4,928.3,928.2,926.0,924.9
4,5,924.2,926.2,926.0,923.7,922.6


In [10]:
df = df.set_index('Day/Time(UTC)').stack().reset_index().set_index('Day/Time(UTC)')
df.index = pd.to_datetime([f'Jan {x}, 2016 {y}' for x, y in zip(df.index, df['level_1'])])
df.drop('level_1', axis=1, inplace=True)
df.columns = ['QFE']
df.head()

,QFE
2016-01-01 05:45:00,927.0
2016-01-01 08:45:00,929.3
2016-01-01 11:45:00,930.0
2016-01-01 14:45:00,927.0
2016-01-01 17:45:00,927.3


In [11]:
main_df['QFE'] = df['QFE']
main_df.head()

,Day/Time(UTC),05:45:00,08:45:00,11:45:00,14:45:00,17:45:00,dew_pt,QFE
0,1,6,7.0,9,8,7,NaN,NaN
1,2,6,7.0,7,8,7,NaN,NaN
2,3,6,7.0,8,8,6,NaN,NaN
3,4,6,7.0,7,7,7,NaN,NaN
4,5,6,6.0,6,7,7,NaN,NaN


# For Cloud

In [12]:
df = pd.read_excel(xls, sheet_name='cloud', skiprows=3, header=[0, 1])
df.head()

Day/Time(UTC) 05:45:00    08:45:00    11:45:00    14:45:00    17:45:00  \
  Unnamed: 0_level_1       LC TC       LC TC       LC TC       LC TC       LC   
0                  1        1  1        4  6        2  2        1  3        1   
1                  2        1  1        1  1        1  1        2  2        1   
2                  3        1  1        1  1        1  1        2  4        2   
3                  4        1  1        1  1        1  1        3  6        1   
4                  5        1  1        2  5        2  6        2  2        1   

      
  TC  
0  2  
1  1  
2  5  
3  2  
4  3

In [13]:
df = df.set_index(('Day/Time(UTC)', 'Unnamed: 0_level_1')).stack(level=0).reset_index()
df = df.set_index(('Day/Time(UTC)', 'Unnamed: 0_level_1'))
df.index = pd.to_datetime([f'Jan {x}, 2016 {y}' for x, y in zip(df.index, df['level_1'])])
df.drop('level_1', axis=1, inplace=True)
df.columns = ['LC', 'TC']
df.head()

,LC,TC
2016-01-01 05:45:00,1,1
2016-01-01 08:45:00,4,6
2016-01-01 11:45:00,2,2
2016-01-01 14:45:00,1,3
2016-01-01 17:45:00,1,2


In [14]:
main_df['LC'] = df['LC']
main_df['TC'] = df['TC']
main_df.head()

,Day/Time(UTC),05:45:00,08:45:00,11:45:00,14:45:00,17:45:00,dew_pt,QFE,LC,TC
0,1,6,7.0,9,8,7,NaN,NaN,NaN,NaN
1,2,6,7.0,7,8,7,NaN,NaN,NaN,NaN
2,3,6,7.0,8,8,6,NaN,NaN,NaN,NaN
3,4,6,7.0,7,7,7,NaN,NaN,NaN,NaN
4,5,6,6.0,6,7,7,NaN,NaN,NaN,NaN


# For the Temperature

In [15]:
df = pd.read_excel(xls, sheet_name='Temperature', skiprows=2, header=[0, 1]).iloc[:, 0:16]
df.head()

Day 05:45:00          08:45:00           11:45:00            \
  Unnamed: 0_level_1      DRY  WET  RH      DRY   WET  RH      DRY   WET  RH   
0                  1      8.7  8.0  92     10.2   9.2  89     17.6  12.2  54   
1                  2      8.6  8.2  95     10.2   9.4  91     18.5  12.8  53   
2                  3      8.4  8.0  95      9.8   8.8  88     18.2  13.2  58   
3                  4      8.8  8.6  98     10.0   9.2  91     18.4  13.3  57   
4                  5      8.2  7.6  93     11.4  10.2  87     18.0  13.0  58   

  14:45:00           17:45:00              
       DRY   WET  RH      DRY   WET    RH  
0     20.8  14.0  46     16.4  12.9  67.0  
1     21.6  14.5  47     16.7  13.0  66.0  
2     20.2  14.0  50     17.3  12.8  60.0  
3     20.2  13.7  49     17.3  12.8  60.0  
4     20.7  14.6  53     17.2  13.7  68.0

In [16]:
df = df.set_index(('Day', 'Unnamed: 0_level_1')).stack(level=0).reset_index()
df = df.set_index(('Day', 'Unnamed: 0_level_1'))
df.index = pd.to_datetime([f'Jan {x}, 2016 {y}' for x, y in zip(df.index, df['level_1'])])
df.drop('level_1', axis=1, inplace=True)
df.columns = ['DRY', 'RH', 'WET']
df.head()

,DRY,RH,WET
2016-01-01 05:45:00,8.7,92.0,8.0
2016-01-01 08:45:00,10.2,89.0,9.2
2016-01-01 11:45:00,17.6,54.0,12.2
2016-01-01 14:45:00,20.8,46.0,14.0
2016-01-01 17:45:00,16.4,67.0,12.9


In [17]:
main_df['DRY'] = df['DRY']
main_df['RH'] = df['RH']
main_df['WET'] = df['WET']
main_df.head()

,Day/Time(UTC),05:45:00,08:45:00,11:45:00,14:45:00,17:45:00,dew_pt,QFE,LC,TC,DRY,RH,WET
0,1,6,7.0,9,8,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,6,7.0,7,8,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,6,7.0,8,8,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,6,7.0,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,6,6.0,6,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Now we have the required df for the month

In [18]:
main_df.to_pickle('./dumps/Jan2016.pkl')
main_df

,Day/Time(UTC),05:45:00,08:45:00,11:45:00,14:45:00,17:45:00,dew_pt,QFE,LC,TC,DRY,RH,WET
0,1,6,7.0,9,8,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,6,7.0,7,8,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,6,7.0,8,8,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,6,7.0,7,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,6,6.0,6,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,6,5.0,6,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,5,1.0,5,7,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,3,2.0,5,6,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,3,3.0,5,6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,3,2.0,5,5,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Now do, the above process for all the months

In [19]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month2int = {month:i+1 for i, month in enumerate(months)}
int2month = {i+1:month for i, month in enumerate(months)}

In [20]:
class IntegrityError(Exception):
    pass

In [21]:
def check_df_integrity(df, sheet):
    if len(df) not in {140, 145, 150, 155}:
        raise IntegrityError(f'Length of dataframe is not standard: {sheet}')
    if df.index[0].hour != 5:
        raise IntegrityError(f'File was parsed wrong way: {sheet}')

In [22]:
def get_visibility(file, month, year):
    df = pd.read_excel(file, sheet_name='visibility', skiprows=3)
    df = df.set_index('Day/Time(UTC)').stack().reset_index().set_index('Day/Time(UTC)')
    df.index = pd.to_datetime([f'{month} {x}, {year} {y}' for x, y in zip(df.index, df['level_1'])])
    df.drop('level_1', axis=1, inplace=True)
    df.columns = ['visibility']
    check_df_integrity(df, 'visibility')
    return df

In [23]:
def get_dew_point(file, month, year):
    df = pd.read_excel(file, sheet_name='Dew pt', skiprows=3)
    df = df.set_index('Day/Time(UTC)').stack().reset_index().set_index('Day/Time(UTC)')
    df.index = pd.to_datetime([f'{month} {x}, {year} {y}' for x, y in zip(df.index, df['level_1'])])
    df.drop('level_1', axis=1, inplace=True)
    df.columns = ['dew_pt']
    check_df_integrity(df, 'dew_pt')
    return df

In [24]:
def get_qfe(file, month, year):
    df = pd.read_excel(file, sheet_name='QFE', skiprows=3)
    df = df.dropna(axis=0).set_index('Day/Time(UTC)').stack().reset_index().set_index('Day/Time(UTC)')
    df.index = pd.to_datetime([f'{month} {x}, {year} {y}' for x, y in zip(df.index, df['level_1'])])
    df.drop('level_1', axis=1, inplace=True)
    df.columns = ['QFE']
    check_df_integrity(df, 'qfe')
    return df.astype(float)

In [25]:
def get_cloud(file, month, year):
    df = pd.read_excel(file, sheet_name='cloud', skiprows=3, header=[0, 1])
    df = df.set_index(('Day/Time(UTC)', 'Unnamed: 0_level_1')).stack(level=0).reset_index()
    df = df.set_index(('Day/Time(UTC)', 'Unnamed: 0_level_1'))
    df.index = pd.to_datetime([f'{month} {x}, {year} {y}' for x, y in zip(df.index, df['level_1'])])
    df.drop('level_1', axis=1, inplace=True)
    df.columns = ['LC', 'TC']
    check_df_integrity(df, 'cloud')
    return df

In [26]:
def get_temperature(file, month, year):
    df = pd.read_excel(file, sheet_name='Temperature', skiprows=2, header=[0, 1]).iloc[:, 0:16]
    df = df.set_index(('Day', 'Unnamed: 0_level_1')).stack(level=0).reset_index()
    df = df.set_index(('Day', 'Unnamed: 0_level_1'))
    df.index = pd.to_datetime([f'{month} {x}, {year} {y}' for x, y in zip(df.index, df['level_1'])])
    df.drop('level_1', axis=1, inplace=True)
    df.columns = ['DRY', 'RH', 'WET']
    check_df_integrity(df, 'temperature')
    return df

In [27]:
def get_main_df(file):
    xls = pd.ExcelFile(file)
    month, year = (file.split('/')[3][:3], file.split()[1][:4])
    df_visibility = get_visibility(xls, month, year)
    df_dew_point = get_dew_point(xls, month, year)
    df_qfe = get_qfe(xls, month, year)
    df_cloud = get_cloud(xls, month, year)
    df_temperature = get_temperature(xls, month, year)
    main_df = pd.concat(
    [
        df_visibility,
        df_dew_point,
        df_qfe,
        df_cloud,
        df_temperature
    ], axis=1)
    main_df['month'] = month2int[month]
    return main_df

In [28]:
PATH = './data'
folders = os.listdir('data')

In [29]:
final_df = pd.DataFrame()
for folder in folders:
    filepath = os.path.join(PATH, folder)
    for file in os.listdir(filepath):
        if file.endswith('.xlsx'):
            try:
                final_df = pd.concat([final_df, get_main_df(os.path.join(filepath, file))])
            except IntegrityError as e:
                print(f'INTEGRITY-ERROR -> {file}: {e}')
            except Exception as e:
                print(f'{file}: {e}')

In [30]:
final_df = final_df.sort_index()
final_df

,visibility,dew_pt,QFE,LC,TC,DRY,RH,WET,month
2016-01-01 05:45:00,6.0,7.4,927.0,1,1,8.7,92.0,8.0,1
2016-01-01 08:45:00,7.0,8.4,929.3,4,6,10.2,89.0,9.2,1
2016-01-01 11:45:00,9.0,7.8,930.0,2,2,17.6,54.0,12.2,1
2016-01-01 14:45:00,8.0,8.8,927.0,1,3,20.8,46.0,14.0,1
2016-01-01 17:45:00,7.0,10.3,927.3,1,2,16.4,67.0,12.9,1
2016-01-02 05:45:00,6.0,7.9,927.3,1,1,8.6,95.0,8.2,1
2016-01-02 08:45:00,7.0,8.8,929.1,1,1,10.2,91.0,9.4,1
2016-01-02 11:45:00,7.0,8.7,928.4,1,1,18.5,53.0,12.8,1
2016-01-02 14:45:00,8.0,9.8,925.7,2,2,21.6,47.0,14.5,1
2016-01-02 17:45:00,7.0,10.2,924.6,1,1,16.7,66.0,13.0,1


# Save dataframe to disk

In [31]:
final_df.to_pickle('./dumps/df.pkl')